<a href="https://colab.research.google.com/github/meefs/art/blob/master/style_transfer_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# style-transfer-pytorch

An implementation of neural style transfer ([A Neural Algorithm of Artistic Style](https://arxiv.org/abs/1508.06576)) in PyTorch, supporting CPUs and Nvidia GPUs. It does automatic multi-scale (coarse-to-fine) stylization to produce high-quality high resolution stylizations, even up to print resolution if the GPUs have sufficient memory.

See [the GitHub repo](https://github.com/crowsonkb/style-transfer-pytorch) for more information.

In [1]:
!nvidia-smi
!git clone https://github.com/crowsonkb/style-transfer-pytorch
!pip install -e ./style-transfer-pytorch

Sat Oct 30 18:03:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!curl -OL 'https://raw.githubusercontent.com/jcjohnson/neural-style/master/examples/inputs/golden_gate.jpg'
!curl -OL 'https://raw.githubusercontent.com/jcjohnson/neural-style/master/examples/inputs/starry_night.jpg'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   99k  100   99k    0     0   536k      0 --:--:-- --:--:-- --:--:--  536k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  307k  100  307k    0     0  1207k      0 --:--:-- --:--:-- --:--:-- 1207k


In [ ]:
!style_transfer -h 

usage: style_transfer [-h] [--output OUTPUT]
                      [--style-weights STYLE_WEIGHT [STYLE_WEIGHT ...]]
                      [--devices DEVICES [DEVICES ...]]
                      [--random-seed RANDOM_SEED]
                      [--content-weight CONTENT_WEIGHT]
                      [--tv-weight TV_WEIGHT] [--min-scale MIN_SCALE]
                      [--end-scale END_SCALE] [--iterations ITERATIONS]
                      [--initial-iterations INITIAL_ITERATIONS]
                      [--save-every SAVE_EVERY] [--step-size STEP_SIZE]
                      [--avg-decay AVG_DECAY]
                      [--init {content,gray,uniform,style_mean}]
                      [--style-scale-fac STYLE_SCALE_FAC]
                      [--style-size STYLE_SIZE] [--pooling {max,average,l2}]
                      [--proof PROOF] [--web] [--host HOST] [--port PORT]
                      [--browser [BROWSER]]
                      content style [style ...]

Neural style transfer (https:/

In [ ]:
!style_transfer \
Diana.jpg \
giuseppe_arcimboldo-spring-1563-obelisk-art-history.jpeg \
 -s 1024 \
 -cw 0.00105 \
 --initial-iterations 2000 \
 --min-scale 256 \
 --step-size 0.02 \
 --style-scale-fac 1.0

Using devices: cuda:0
GPU 0 type: Tesla T4 (compute 7.5)
GPU 0 RAM: 15110 MB
Loading model...
Processing content image (199x256)...
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Processing style image (192x256)...
Size: 199x256, iteration: 1, loss: 0.427665
Size: 199x256, iteration: 2, loss: 0.379682
Size: 199x256, iteration: 3, loss: 0.343952
Size: 199x256, iteration: 4, loss: 0.291449
Size: 199x256, iteration: 5, loss: 0.276766
Size: 199x256, iteration: 6, loss: 0.259016
Size: 199x256, iteration: 7, loss: 0.248495
Size: 199x256, iteration: 8, loss: 0.238214
Size: 199x256, iteration: 9, loss: 0.229848
Size: 199x256, iteration: 10, lo

In [ ]:
import IPython

IPython.display.Image('out.png')